# COMP5046 - Natural Language Processing Assignment 2: QandA Bot
Brandon James Temple  
BTEM3257  
460372949  

# 1. Data Preprocessing

In [0]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import pandas as pd


# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download all three Microsoft BotBuilder Personality Chat Datasets on Google Colab virtual server

id = '1SXoGbD9WZHwhpqR-cBw7-8_7Ri06nIb6'
downloaded = drive.CreateFile({'id':id }) 
downloaded.GetContentFile('WikiQA-train.tsv') 

id = '1TwuDSxlcAFDnTRpF-GRvqRXoR_UsJznH'
downloaded = drive.CreateFile({'id':id }) 
downloaded.GetContentFile('WikiQA-test.tsv') 

     |████████████████████████████████| 993kB 5.1MB/s 


In [0]:
import pandas as pd

# train.tsv
df_train=pd.read_csv('WikiQA-train.tsv', sep='\t')
# test.tsv
df_test=pd.read_csv('WikiQA-test.tsv', sep='\t')

In [0]:
train_dataset = []
test_dataset = []
max_document_len = 0
max_question_len = 0
max_document_sents = 0

In [0]:
# Training
q_id = -1
a_i_count = 0
no_answer = True
curr_triplet = {}
for row in df_train.iterrows():
    curr_q_id = row[1]['QuestionID']
    if q_id == -1:
        q_id = curr_q_id
    if curr_q_id != q_id:
        if no_answer:
            curr_triplet['answer'] = -1
        if len(curr_triplet['question']) > max_question_len:
            max_question_len = len(curr_triplet['question'])
        if #sum doc lengths > max_document_len:
            max_document_len = # sum of doc sent lengths
        if len(curr_triplet['document']) > max_document_sents:
            max_document_sents = len(curr_triplet['document'])
        train_dataset.append(curr_triplet)
        curr_triplet = {}
        a_i_count = 0
        no_answer = True
        q_id = curr_q_id
    question =  row[1]['Question']
    document =  row[1]['Sentence']
    is_answer =  row[1]['Label']
    if not ('question' in curr_triplet):
        curr_triplet['question'] = question.replace('-', ' ').replace('—', ', ')
    if not ('document' in curr_triplet):
        curr_triplet['document'] = [document.replace('-', ' ').replace('—', ', ')]
    else:
        curr_triplet['document'].append(document.replace('-', ' ').replace('—', ', '))
    if is_answer == 1:
        curr_triplet['answer'] = a_i_count
        no_answer = False
    else:
        a_i_count += 1

In [0]:
# Testing
q_id = -1
a_i_count = 0
no_answer = True
curr_triplet = {}
for row in df_train.iterrows():
    curr_q_id = row[1]['QuestionID']
    if q_id == -1:
        q_id = curr_q_id
    if curr_q_id != q_id:
        if no_answer:
            curr_triplet['answer'] = -1
        test_dataset.append(curr_triplet)
        curr_triplet = {}
        a_i_count = 0
        no_answer = True
        q_id = curr_q_id
    question =  row[1]['Question']
    document =  row[1]['Sentence']
    is_answer =  row[1]['Label']
    if not ('question' in curr_triplet):
        curr_triplet['question'] = question.replace('-', ' ').replace('—', ', ')
    if not ('document' in curr_triplet):
        curr_triplet['document'] = [document.replace('-', ' ').replace('—', ', ')]
    else:
        curr_triplet['document'].append(document.replace('-', ' ').replace('—', ', '))
    if is_answer == 1:
        curr_triplet['answer'] = a_i_count
        no_answer = False
    else:
        a_i_count += 1

In [0]:
# combine datasets
full_dataset = train_dataset + test_dataset

# clear unneeded data from memory
del df_train
del df_test
del train_dataset
del test_dataset

# 2. Q&A Framework

## 2.1 Word Embedding and Feature Extraction

### 2.1.1 Word Embedding 

**WARNING**: This could max out the ram but it shouldn't

In [0]:
! wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
! gunzip cc.en.300.bin.gz

--2019-06-10 09:26:24--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.6.166, 104.20.22.166, 2606:4700:10::6814:16a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.6.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: ‘cc.en.300.bin.gz’

cc.en.300.bin.gz    100%[===================>]   4.19G  33.6MB/s    in 1m 50s  

2019-06-10 09:28:15 (38.9 MB/s) - ‘cc.en.300.bin.gz’ saved [4503593528/4503593528]



In [0]:
from gensim.models import FastText

word_embedding = FastText.load_fasttext_format('cc.en.300.bin')

### 2.1.2 Feature Extraction

#### 2.1.2.1 POS Tagging

In [0]:
# NLTK Imports and downloads
import nltk

nltk.download('punkt')
nltk.download('brown')

from nltk.tag import hmm
from nltk.corpus import brown

# Train a tagger
trainer = hmm.HiddenMarkovModelTrainer()
trained_tagger = trainer.train_supervised(brown.tagged_sents())

# Define function to tag sentences
# NEED TO BREAK IN SENTENCE AND WHOLE STRING
def pos_tag(tokens):
    tags = []
    for tagged_word in trained_tagger.tag(tokens):
        tag = tagged_word[1]
        if tag.startswith('NN'):
            tags.append([1,0,0,0])
        elif tag.startswith('JJ'):
            tags.append([0,1,0,0])
        elif tag.startswith('RB'):
            tags.append([0,0,1,0])
        elif tag.startswith('VB'):
            tags.append([0,0,0,1])
        else:
            tags.append([0,0,0,0])
    return tags

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


#### 2.1.2.2 TF-IDF
The Document in whole is treated as the Corpus and each sentence a Document for frequency purposes

In [0]:
# Collections and math import
import collections
import math

# Define function to calculate word frequencies
def tfidf(sentences):
    whole_scores = []
    N = len(sentences)
    corpus = sum(sentences, [])
    corpus_counter = collections.Counter(corpus)
    sentence_counters = []
    for s in sentences:
        sentence_counters.append(collections.Counter(s))
    for i in range(len(sentences)):
        current_counter = sentence_counters[i]
        for word in sentences[i]:
            tf = current_counter[word]
            df = 0
            for counter in sentence_counters:
                if counter[word] > 0:
                    df += 1
            weight = tf * math.log(N/df)
            whole_scores.append(weight)
    return whole_scores
            
    

#### 2.1.2.3 Named Entity Recognition

In [0]:
!pip install -U spacy
!python -m spacy download en

     |████████████████████████████████| 29.8MB 1.5MB/s 
     |████████████████████████████████| 2.1MB 39.3MB/s 
     |████████████████████████████████| 184kB 44.2MB/s 
     |████████████████████████████████| 3.2MB 37.1MB/s 
  Found existing installation: thinc 6.12.1
    Uninstalling thinc-6.12.1:
      Successfully uninstalled thinc-6.12.1
  Found existing installation: spacy 2.0.18
    Uninstalling spacy-2.0.18:
      Successfully uninstalled spacy-2.0.18
     |████████████████████████████████| 11.1MB 779kB/s 
  Stored in directory: /tmp/pip-ephem-wheel-cache-qlhjs0tt/wheels/39/ea/3b/507f7df78be8631a7a3d7090962194cf55bc1158572c0be77f
Successfully built en-core-web-sm
  Found existing installation: en-core-web-sm 2.0.0
    Uninstalling en-core-web-sm-2.0.0:
      Successfully uninstalled en-core-web-sm-2.0.0
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/

Tests for Person, Organistion, Date, NORP, and GPE

In [0]:
# THIS MIGHT TOKENISE THINGS DIFFERENTLY WHICH WILL CAUSE PROBLEMS
# Import spacy and en_core
import spacy
import en_core_web_sm

# Load NER model
recogniser = en_core_web_sm.load()

def ner(tokens):
    sentence = recogniser(tokens)
    entities = [(x, x.ent_iob_, x.ent_type_) for x in sentence]
    tags = []
    for entity in entities:
        tag = []
        # is it an entity?
        if entity[1] == 'O':
            tag += [0]
        else:
            tag += [1]
        
        # if entity, is it I or B?
        if entity[1] == 'B':
            tag += [1, 0]
        elif entity[1] == 'I':
            tag += [0, 1]
        else:
            tag += [0, 0]
            
        # if entity, what type?
        if entity[2] == 'PERSON':
            tag += [1,0,0,0,0,0]
        elif entity[2] == 'DATE':
            tag += [0,1,0,0,0,0]
        elif entity[2] == 'ORG':
            tag += [0,0,1,0,0,0]
        elif entity[2] == 'NORP':
            tag += [0,0,0,1,0,0]
        elif entity[2] == 'GPE':
            tag += [0,0,0,0,1,0]
        elif entity[2] == '':
            tag += [0,0,0,0,0,0]
        else:
            tag += [0,0,0,0,0,1]
        tags.append(tag)
    return tags

#### 2.1.2.4 Dependancy Path

In [0]:
!pip install -U spacy
!python -m spacy download en

In [0]:
# Import spacy
import spacy

# Download parser
pather = spacy.load("en_core_web_sm")

def dependancy_path(sentences):
    heads = []
    for sentence in sentences:
        parse = pather(sentence)
        for token in parse:
            if token.dep_ == 'ROOT':
                heads.append([-1])
            else:
                heads.append([token.head.i])
    return heads

#### 2.1.2.5 Word Matching

In [0]:
# Import NLTK, and download Lemmatizer
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')

lemmatiser = WordNetLemmatizer()
def word_match(sentence, question):
    matches = []
    # lemmatise question
    question_lemmatised = [lemmatiser.lemmatize(x.lower()) for x in question]
    # check is lemmatised word in question
    for word in sentence:
        if lemmatiser.lemmatize(word.lower()) in question_lemmatised:
            matches.append(1)
        else:
            matches.append(0)
    return matches

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### 2.1.3 Processing

You can change these variables to alter which features are used when not using the automated evaluation algorithms

In [0]:
use_pos = False
use_tfidf = False
use_ner = False
use_dpath = False
use_matching = False

## 2.2 Sequence Model

### 2.2.1 Batch Generation

In [0]:
import re
import numpy as np

def generate_batch(dataset):
    document_batch = []
    question_batch = []
    answer_batch = []
    documents = []
    questions = []
    answer_keys = []
    doc_vec_len = 0
    for i in range(len(dataset)):
        data = dataset[i]
        question = data['question']
        document_sentences = data['document']
        document_whole = ''.join(document_sentences)
        
        # tokenise
        question_tokens = question.split(' ')
        document_whole_tokens = document_whole.split(' ')
        document_sentence_tokens = []
        for sentence in document_sentences:
            document_sentence_tokens.append(sentence.split(' '))
            
        
        # make token vectors
        question_vectors = []
        for token in question_tokens:
            word = token.lower()
            word = re.sub(r'[^\w\s]','', word)
            if word == '':
                continue
            try:
                question_vectors.append(list(word_embedding[word]))
            except:
                question_vectors.append([0]*300)
        document_vectors = []
        for token in document_whole_tokens:
            word = token.lower()
            word = re.sub(r'[^\w\s]','', word)
            if word == '':
                continue
            try:
                document_vectors.append(list(word_embedding[word]))
            except:
                question_vectors.append([0]*300)
        print ('\r %d of %d processing...' % (i+1, len(dataset)), end='')
        
        # extract features
        if use_pos:
            tags = pos_tag(document_whole)
            for i in range(len(document_vectors)):
                document_vectors[i] += tags[i]
                
        if use_tfidf:
            freqs = tfidf(document_sentence_tokens)
            for i in range(len(document_vectors)):
                document_vectors[i] += [freqs[i]]
                
        if use_ner:
            tags = ner(document_whole)
            for i in range(len(document_vectors)):
                document_vectors[i] += tags[i]
                
        if use_dpath:
            heads = dependancy_path(document_sentences)
            for i in range(len(document_vectors)):
                document_vectors[i] += heads[i]
                
        if use_matching:
            match = word_match(document_whole_tokens, question_tokens)
            for i in range(len(document_vectors)):
                document_vectors[i] += [match[i]]
        
        doc_vec_len = len(document_vectors[0])
        for vector in document_vectors:
            vector = np.array(vector)
        for vector in question_vectors:
            vector = np.array(vector)
        
        # add to batches
        documents.append(document_vectors)
        questions.append(question_vectors)
        answer_keys.append(data['answer'])
        
    # Pad Doc and Q Batches:
    for document in documents:
        difference = max_document_len - len(document)
        for _ in range(difference):
            document.append(np.array([0]*doc_vec_len))
        document_batch.append(np.array(document))
        
    for question in questions:
        difference = max_question_len - len(question)
        for _ in range(difference):
            question.append(np.array([0]*300))
        question_batch.append(np.array(question))
        
    # transform answer to 1hot
    for answer in answer_keys:
        i = np.eye(max_document_sents+1)
        vector = i[answer]
        answer_batch.append(vector)
    return np.array(document_batch), np.array(question_batch), np.array(answer_batch)
            

### 2.2.2 Build Model

In [0]:
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import Dot
from keras.layers import Dense
from keras.layers import Softmax
from keras.layers import Flatten
import keras.backend as K


# precision and recall code are as they originally were in Keras, see:
# Title: Keras Source Code
# Author: carlthome
# Date: 4 Nov 2016
# Commit: https://github.com/keras-team/keras/commit/2b51317be82d4420169d2cc79dc4443028417911#diff-7b49e1c42728a58a9d08643a79f44cd4

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def fscore(y_true, y_pred):
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2*((p*r)/(p+r))

def create_model(feature_len, answer_len, doc_len, q_len):
    # document
    document_input = Input(shape=(doc_len,feature_len), name='document_embeddingsfeatures')
    document_bilstm = Bidirectional(LSTM(answer_len, return_sequences=True), name ='document_bilstm', merge_mode='sum')(document_input)
    
    # question
    question_input = Input(shape=(q_len,300), name='question_embeddings')
    question_bilstm = Bidirectional(LSTM(answer_len, return_sequences=True), name='question_bilstm', merge_mode='sum')(question_input)
    
    # attention
    attention = Dot(name='attention_dot_score', axes=(2,2))([document_bilstm, question_bilstm])
    reshape = Flatten()(attention)
    dense = Dense(answer_len)(reshape)
    softmax = Softmax(axis=1, name='attenion_softmax')(dense)
    
    model = Model(inputs = [document_input, question_input], outputs = softmax)
    model.compile(
        optimizer='adam',
        loss='mean_squared_error',
        metrics=['acc', precision, recall, fscore],
    )
    return model


Using TensorFlow backend.


### 2.2.3 Train Model

In [0]:
def train_model(model, document_batch, question_batch, answer_batch, epoch_num):
    model.fit([document_batch, question_batch], answer_batch, batch_size=10, epochs=epoch_num, verbose=1)

## 2.3 Evaluation

### 2.3.1 Which Features

In [0]:
import warnings
warnings.filterwarnings('ignore')

use_matching = False

# These represent the possible feature configs
permutations= [[True, False, False, False],
               [False, True, False, False],
               [False, False, True, False],
               [False, False, False, True],
               [True, True, False, False],
               [False, True, True, False],
               [False, False, True, True],
               [True, False, False, True],
               [True, False, True, False],
               [False, True, False, True],
               [True, True, True, False],
               [False, True, True, True],
               [True, False, True, True],
               [True, True, False, True],
               [True, True, True, True]]

for p in permutations:
    # Unpack feature config
    use_pos, use_tfidf, use_ner, use_dpath = p
    
    print('pos: {}\ttfidf: {}\tner: {}\tdpath: {}\tmatching: {}\n'.format(use_pos, use_tfidf, use_ner, use_dpath, use_matching))
    print('generating batch...\n')
    document_batch, question_batch, answer_batch = generate_batch(train_dataset[:300])
    
    # Train
    print('training...\n')
    model = create_model(document_batch.shape[2], answer_batch.shape[1], document_batch.shape[1], question_batch.shape[1])
    train_model(model, document_batch[:200], question_batch[:200], answer_batch[:200], 50)
    
    # Test
    print('testing')
    print(model.evaluate([document_batch[200:], question_batch[200:]], [answer_batch[200:]], batch_size=10))

pos: True	tfidf: False	ner: False	dpath: False	matching: False

generating batch...

 300 of 300 processing...training...

Epoch 1/50
200/200 [==============================] - 24s 120ms/step - loss: 0.0245 - acc: 0.5900 - precision: 0.4175 - recall: 0.3500 - fscore: nan
Epoch 2/50
200/200 [==============================] - 17s 87ms/step - loss: 0.0228 - acc: 0.5950 - precision: 0.5950 - recall: 0.5950 - fscore: 0.5950
Epoch 3/50
200/200 [==============================] - 17s 86ms/step - loss: 0.0187 - acc: 0.6400 - precision: 0.6756 - recall: 0.5350 - fscore: 0.5913
Epoch 4/50
200/200 [==============================] - 17s 86ms/step - loss: 0.0178 - acc: 0.6400 - precision: 0.6759 - recall: 0.5850 - fscore: 0.6234
Epoch 5/50
200/200 [==============================] - 17s 86ms/step - loss: 0.0164 - acc: 0.6500 - precision: 0.7425 - recall: 0.6100 - fscore: 0.6661
Epoch 6/50
200/200 [==============================] - 17s 86ms/step - loss: 0.0150 - acc: 0.6800 - precision: 0.7685 - recal

### 2.3.2 Full Model

#### 2.3.2.1 Build and Train

In [0]:
# Declare features for final model
use_pos = False
use_ner = True
use_dpath = True
use_tfidf = False
use_matching = False

# Create batch for final model
document_batch, question_batch, answer_batch = generate_batch(full_dataset[:750])

# Clear fulldataset since its nolonger needed and saves memory
del full_dataset

# Create and Train on training data
model = create_model(document_batch.shape[2], answer_batch.shape[1], document_batch.shape[1], question_batch.shape[1])
train_model(model, document_batch[:650], question_batch[:650], answer_batch[:650], 100)

# Evaluate
print(model.evaluate([document_batch[650:], question_batch[650:]], [answer_batch[650:]], batch_size=10))

# Save model
model.save('model.h5')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:53: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


 750 of 750 processing...WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
650/650 [==============================] - 66s 102ms/step - loss: 0.0235 - acc: 0.5692 - precision: 0.6019 - recall: 0.4646 - fscore: nan
Epoch 2/100
650/650 [==============================] - 62s 95ms/step - loss: 0.0207 - acc: 0.6092 - precision: 0.6573 - recall: 0.5354 - fscore: 0.5844
Epoch 3/100
650/650 [==============================] - 62s 96ms/step - loss: 0.0186 - acc: 0.6400 - precision: 0.6891 - recall: 0.5569 - fscore: 0.6107
Epoch 4/100
650/650 [==============================] - 63s 97ms/step - loss: 0.0170 - acc: 0.6646 - precision: 0.7353 - recall: 0.5769 - fscore: 0.6406
Epoch 5/100
650/650 [======

#### 2.3.2.2 Load Model

In [0]:
from keras.models import load_model

# Declare features for final model
use_pos = False
use_ner = True
use_dpath = True
use_tfidf = False
use_matching = False

# Create Batch
document_batch, question_batch, answer_batch = generate_batch(full_dataset[:750])
del full_dataset

# Load Model
model = load_model('model.h5')

# Evaluate
print(model.evaluate([document_batch[650:], question_batch[650:]], [answer_batch[650:]], batch_size=10))